In [2]:
# ========== 1) SETUP ==========
import numpy as np
import pandas as pd

# --- I/O (adjust paths if needed) ---
dividend_df = pd.read_csv("Dividend_Dataset.csv", parse_dates=["Announcement_Date"])
returns_df  = pd.read_csv("Merged_Data.csv",    parse_dates=["Date"])

# Expected columns:
# dividend_df: Announcement_Date, Ticker, [Dividend_Type, Dividend_Amount, Company_Name, ...]
# returns_df : Date, Ticker, Stock_Returns, Market_Returns

# --- Parameters (trading-day based) ---
EST_LENGTH    = 110   # use exactly 110 clean obs
BUFFER_LENGTH = 10    # buffer right before event (excluded from estimation)
EVENT_K       = 5     # event window [-5, +5]

# --- Safety: types, sort, dedupe events ---
returns_df["Date"] = pd.to_datetime(returns_df["Date"])
dividend_df["Announcement_Date"] = pd.to_datetime(dividend_df["Announcement_Date"])
returns_df = returns_df.sort_values(["Ticker","Date"]).reset_index(drop=True)
dividend_df = (dividend_df
               .sort_values(["Ticker","Announcement_Date"])
               .drop_duplicates(subset=["Ticker","Announcement_Date"])
               .reset_index(drop=True))

In [3]:
# ========== 2) ESTIMATION ONLY (STRICT) ==========
alpha_beta_rows = []
drop_rows_est = []

def log_drop_est(ticker, D, reason, detail=None):
    drop_rows_est.append({
        "Ticker": ticker,
        "Event_Date": pd.to_datetime(D),
        "Reason": reason,
        "Detail": detail
    })

need = EST_LENGTH + BUFFER_LENGTH

for _, ev in dividend_df[["Ticker","Announcement_Date"]].dropna().iterrows():
    ticker = ev["Ticker"]; D = ev["Announcement_Date"]
    firm = returns_df[returns_df["Ticker"] == ticker]
    if firm.empty:
        log_drop_est(ticker, D, "no_returns_for_ticker")
        continue

    pre_all = firm[firm["Date"] < D]
    if pre_all.shape[0] < need:
        log_drop_est(ticker, D, "insufficient_pre_event_history",
                     detail=f"Have {pre_all.shape[0]} < required {need}")
        continue

    pre = pre_all.tail(need).reset_index(drop=True)
    est = pre.iloc[:EST_LENGTH].copy()

    Ri = est["Stock_Returns"].to_numpy()
    Rm = est["Market_Returns"].to_numpy()

    # STRICT: must be completely clean
    if np.isnan(Ri).any() or np.isnan(Rm).any():
        log_drop_est(ticker, D, "nan_in_estimation_window",
                     detail=f"NaNs -> Stock_Returns:{int(np.isnan(Ri).sum())}, Market_Returns:{int(np.isnan(Rm).sum())}")
        continue

    var_m = np.var(Rm, ddof=1)
    if (not np.isfinite(var_m)) or (var_m == 0):
        log_drop_est(ticker, D, "zero_or_nan_market_variance_in_estimation_window",
                     detail=f"var_m={var_m}")
        continue

    cov_im = np.cov(Ri, Rm, ddof=1)[0, 1]
    beta  = cov_im / var_m
    alpha = Ri.mean() - beta * Rm.mean()

    # Optional R2
    fitted = alpha + beta * Rm
    ss_tot = np.sum((Ri - Ri.mean())**2)
    ss_res = np.sum((Ri - fitted)**2)
    R2 = 1 - ss_res / ss_tot if ss_tot > 0 else np.nan

    alpha_beta_rows.append({
        "Ticker": ticker,
        "Event_Date": D,
        "Alpha": float(alpha),
        "Beta": float(beta),
        "R2_optional": float(R2) if np.isfinite(R2) else np.nan,
        "Estimation_Obs": EST_LENGTH
    })

alpha_beta_df = pd.DataFrame(alpha_beta_rows).sort_values(["Ticker","Event_Date"]).reset_index(drop=True)
drop_log_est  = pd.DataFrame(drop_rows_est).sort_values(["Event_Date","Ticker"]).reset_index(drop=True)

# --- Save αβ estimates (requested) ---
alpha_beta_df.to_csv("alpha_beta_by_event.csv", index=False)

print(f"Estimated alphas/betas for {alpha_beta_df.shape[0]} events "
      f"(dropped in estimation: {drop_log_est.shape[0]}).")
print("Saved: alpha_beta_by_event.csv")

Estimated alphas/betas for 702 events (dropped in estimation: 58).
Saved: alpha_beta_by_event.csv


In [4]:
alpha_beta_df

,Ticker,Event_Date,Alpha,Beta,R2_optional,Estimation_Obs
0,ADANIENT.NS,2016-03-09,-0.000245,1.934967,0.371993,110
1,ADANIENT.NS,2017-05-24,0.004341,1.355969,0.051619,110
2,ADANIENT.NS,2018-05-10,0.000159,2.711392,0.334825,110
3,ADANIENT.NS,2019-05-29,-0.002520,1.776056,0.210455,110
4,ADANIENT.NS,2020-03-12,0.004043,1.457345,0.363193,110
...,...,...,...,...,...,...
697,WIPRO.NS,2022-03-25,-0.000004,0.951547,0.367928,110
698,WIPRO.NS,2023-01-13,-0.001342,1.042780,0.425205,110
699,WIPRO.NS,2024-01-12,-0.000001,1.289917,0.316941,110
700,WIPRO.NS,2025-01-17,0.001834,1.184604,0.374806,110


In [5]:
# ========== 3) EVENT WINDOW + AAR/CAAR (same format as before; no significance flags) ==========

panel_rows = []

for _, row in alpha_beta_df.iterrows():
    ticker = row["Ticker"]
    D      = row["Event_Date"]
    alpha  = row["Alpha"]
    beta   = row["Beta"]

    firm = returns_df[returns_df["Ticker"] == ticker]
    if firm.empty:
        continue

    # K trading days before (strictly < D) and 0..+K (>= D)
    preK  = firm[firm["Date"] < D].tail(EVENT_K)
    postK = firm[firm["Date"] >= D].head(EVENT_K + 1)

    # Require exact coverage
    if (len(preK) < EVENT_K) or (len(postK) < (EVENT_K + 1)):
        continue

    preK  = preK.sort_values("Date").assign(event_day_index=np.arange(-EVENT_K, 0, 1))
    postK = postK.sort_values("Date").assign(event_day_index=np.arange(0, EVENT_K + 1, 1))
    window = pd.concat([preK, postK], ignore_index=True)

    # Expected & abnormal returns (Market Model)
    expected = alpha + beta * window["Market_Returns"].to_numpy()
    AR       = window["Stock_Returns"].to_numpy() - expected

    panel_rows.append(pd.DataFrame({
        "Ticker": ticker,
        "event_date": D,
        "trading_date": window["Date"].to_numpy(),
        "event_day_index": window["event_day_index"].to_numpy(),
        "Market_Return": window["Market_Returns"].to_numpy(),
        "Stock_Return": window["Stock_Returns"].to_numpy(),
        "Expected_Return": expected,
        "Abnormal_Return": AR
    }))

sym_panel = (
    pd.concat(panel_rows, ignore_index=True)
    if panel_rows else
    pd.DataFrame(columns=[
        "Ticker","event_date","trading_date","event_day_index",
        "Market_Return","Stock_Return","Expected_Return","Abnormal_Return"
    ])
)

# ---------- AAR (per event day) ----------
if sym_panel.empty:
    aar_table = pd.DataFrame(columns=[
        "Event_Day_Index","Number_of_Events","Average_Abnormal_Return",
        "Standard_Deviation","Standard_Error","t_statistic"
    ])
else:
    aar = (
        sym_panel.groupby("event_day_index")["Abnormal_Return"]
                 .agg(Number_of_Events="count",
                      Average_Abnormal_Return="mean",
                      Standard_Deviation=lambda x: x.std(ddof=1))
                 .reset_index()
                 .sort_values("event_day_index")
    )
    aar["Standard_Error"] = aar["Standard_Deviation"] / np.sqrt(aar["Number_of_Events"])
    aar["t_statistic"]    = aar["Average_Abnormal_Return"] / aar["Standard_Error"]
    aar_table = aar.rename(columns={"event_day_index": "Event_Day_Index"})

# ---------- CAAR over [-K, +K] ----------
if sym_panel.empty:
    caar_summary = pd.DataFrame([{
        "Number_of_Events": 0,
        "CAAR_from_AAR_Sum": np.nan,
        "CAAR_from_Mean_of_CARs": np.nan,
        "Standard_Deviation_of_CARs": np.nan,
        "Standard_Error_of_CARs": np.nan,
        "t_statistic_for_Mean_CAR": np.nan
    }])
    car_by_event = pd.DataFrame(columns=["Ticker","event_date","CAR"])
else:
    caar_from_aar = float(aar_table["Average_Abnormal_Return"].sum())
    car_by_event = (
        sym_panel.groupby(["Ticker","event_date"], as_index=False)
                 .agg(CAR=("Abnormal_Return","sum"))
                 .sort_values(["Ticker","event_date"])
    )
    N_ev   = car_by_event.shape[0]
    caar_mean = float(car_by_event["CAR"].mean()) if N_ev > 0 else np.nan
    caar_sd   = float(car_by_event["CAR"].std(ddof=1)) if N_ev > 1 else np.nan
    caar_se   = (caar_sd / np.sqrt(N_ev)) if N_ev > 1 else np.nan
    caar_t    = (caar_mean / caar_se) if (isinstance(caar_se, float) and caar_se > 0) else np.nan

    caar_summary = pd.DataFrame([{
        "Number_of_Events": N_ev,
        "CAAR_from_AAR_Sum": caar_from_aar,
        "CAAR_from_Mean_of_CARs": caar_mean,
        "Standard_Deviation_of_CARs": caar_sd,
        "Standard_Error_of_CARs": caar_se,
        "t_statistic_for_Mean_CAR": caar_t
    }])

# ---------- OUTPUT (same style as your earlier printout) ----------
n_kept = sym_panel[["Ticker","event_date"]].drop_duplicates().shape[0] if not sym_panel.empty else 0
print(f"Built symmetric panel for [-{EVENT_K}, +{EVENT_K}] with {len(sym_panel)} rows across {n_kept} events.")

print("\nAAR table (pooled by event day):")
print(aar_table.to_string(index=False))

print("\nCAAR summary for [-{0}, +{0}] window:".format(EVENT_K))
print(caar_summary.to_string(index=False))

# ---------- SAVE EXTRA CSVs (but don't print contents) ----------
#sym_panel.to_csv(f"AR_panel_[-{EVENT_K},+{EVENT_K}].csv", index=False)
#aar_table.to_csv(f"AAR_table_[-{EVENT_K},+{EVENT_K}].csv", index=False)
#caar_summary.to_csv(f"CAAR_summary_[-{EVENT_K},+{EVENT_K}].csv", index=False)
car_by_event.to_csv(f"CAR_by_event.csv", index=False)

# ========== MERGE CAR WITH DIVIDEND DATA ==========
print("\nMERGING CAR WITH DIVIDEND DATA")
print("=" * 40)

# Load the dividend dataset 
dividend_df = pd.read_csv("Dividend_Dataset.csv", parse_dates=["Announcement_Date"])
car_df = pd.read_csv("CAR_by_event.csv", parse_dates=["event_date"])

# Rename and merge
car_df = car_df.rename(columns={'event_date': 'Announcement_Date'})
merged_df = pd.merge(
    car_df,
    dividend_df[['Announcement_Date', 'Ticker', 'Dividend_Amount', 'Dividend_Type']],
    on=['Announcement_Date', 'Ticker'],
    how='left'
)

# Rename Dividend_Amount to DPS
merged_df = merged_df.rename(columns={'Dividend_Amount': 'DPS'})

print(f"CAR events: {len(car_df)}")
print(f"Merged events: {len(merged_df)}")
print(f"Events with dividend data: {merged_df['DPS'].notnull().sum()}")
print(f"Events without dividend data: {merged_df['DPS'].isnull().sum()}")

# Save the merged dataset
merged_df.to_csv('merged_dividend_car_data.csv', index=False)
print("Saved: merged_dividend_car_data.csv")

# ========== NOW CONTINUE WITH PRICE MERGE ==========

Built symmetric panel for [-5, +5] with 7183 rows across 653 events.

AAR table (pooled by event day):
 Event_Day_Index  Number_of_Events  Average_Abnormal_Return  Standard_Deviation  Standard_Error  t_statistic
              -5               653                -0.000358            0.016632        0.000651    -0.550451
              -4               653                 0.000642            0.016596        0.000649     0.988550
              -3               653                 0.000163            0.015608        0.000611     0.267362
              -2               653                 0.000010            0.015567        0.000609     0.016326
              -1               653                -0.000391            0.017445        0.000683    -0.573344
               0               653                 0.000834            0.021052        0.000824     1.012865
               1               653                -0.000276            0.029670        0.001161    -0.237695
               2         

In [44]:
import pandas as pd

# Assuming you have both datasets loaded as DataFrames
dividend_df = pd.read_csv('Dividend_Dataset.csv')  # or your file path
car_df = pd.read_csv('CAR_by_event.csv')           # or your file path

# Convert date columns to datetime to ensure proper matching
dividend_df['Announcement_Date'] = pd.to_datetime(dividend_df['Announcement_Date'])
car_df['event_date'] = pd.to_datetime(car_df['event_date'])

# First, rename event_date to Announcement_Date in the car_df
car_df = car_df.rename(columns={'event_date': 'Announcement_Date'})

# Now merge the datasets
merged_df = pd.merge(
    car_df,
    dividend_df[['Announcement_Date', 'Ticker', 'Dividend_Amount', 'Dividend_Type']],
    on=['Announcement_Date', 'Ticker'],
    how='left'
)

# Rename Dividend_Amount to DPS
merged_df = merged_df.rename(columns={'Dividend_Amount': 'DPS'})

# Display the merged dataframe
print(merged_df.head())
print(f"\nMerged dataset shape: {merged_df.shape}")
print(f"\nColumns in merged dataset: {merged_df.columns.tolist()}")

# Check for any missing values in the merged data
print(f"\nMissing values in each column:")
print(merged_df.isnull().sum())

# If you want to save the merged dataset
merged_df.to_csv('merged_dividend_car_data.csv', index=False)

        Ticker Announcement_Date       CAR  DPS Dividend_Type
0  ADANIENT.NS        2016-03-09 -0.016032  0.4       Interim
1  ADANIENT.NS        2017-05-24 -0.108284  0.4         Final
2  ADANIENT.NS        2018-05-10 -0.039030  0.4         Final
3  ADANIENT.NS        2019-05-29 -0.000906  0.4         Final
4  ADANIENT.NS        2020-03-12 -0.135723  1.0       Interim

Merged dataset shape: (657, 5)

Columns in merged dataset: ['Ticker', 'Announcement_Date', 'CAR', 'DPS', 'Dividend_Type']

Missing values in each column:
Ticker               0
Announcement_Date    0
CAR                  0
DPS                  0
Dividend_Type        0
dtype: int64


### Dividend_Yield

In [6]:
import pandas as pd

# Load the stock prices dataset
prices_df = pd.read_csv('nifty50_stock_prices.csv')

# Convert date columns to datetime format
prices_df['Date'] = pd.to_datetime(prices_df['Date'])
merged_df['Announcement_Date'] = pd.to_datetime(merged_df['Announcement_Date'])

# Create a copy to avoid modifying the original during operations
result_df = merged_df.copy()

# Get event date's AdjClose price
event_prices = prices_df[['Date', 'Ticker', 'AdjClose']].copy()
event_prices = event_prices.rename(columns={'AdjClose': 'Event_Date_AdjClose'})

result_df = pd.merge(
    result_df,
    event_prices,
    left_on=['Announcement_Date', 'Ticker'],
    right_on=['Date', 'Ticker'],
    how='left'
).drop('Date', axis=1)

# Create a dictionary of all available trading dates for each ticker
available_dates = {}
for ticker in prices_df['Ticker'].unique():
    ticker_dates = set(prices_df[prices_df['Ticker'] == ticker]['Date'])
    available_dates[ticker] = sorted(ticker_dates)

def get_most_recent_trading_price(ticker, announcement_date):
    """Get the most recent trading day price before announcement date"""
    if ticker not in available_dates:
        return None
    
    # Get all trading dates that are BEFORE the announcement date
    prior_dates = [date for date in available_dates[ticker] if date < announcement_date]
    
    if not prior_dates:
        return None
    
    # Get the most recent trading date
    most_recent_date = max(prior_dates)
    
    # Get the price for that date
    return prices_df[(prices_df['Ticker'] == ticker) & 
                    (prices_df['Date'] == most_recent_date)]['AdjClose'].values[0]

# Get previous trading day's AdjClose price
result_df['Prior_Day_AdjClose'] = result_df.apply(
    lambda row: get_most_recent_trading_price(row['Ticker'], row['Announcement_Date']),
    axis=1
)

# Calculate Dividend Yield (DPS / Prior Trading Day AdjClose Price)
result_df['Dividend_Yield'] = result_df['DPS'] / result_df['Prior_Day_AdjClose']

# Display the results
print("Final DataFrame Shape:", result_df.shape)
print("\nColumns in final dataset:")
print(result_df.columns.tolist())
print(f"\nNew columns added: Event_Date_AdjClose, Prior_Day_AdjClose, Dividend_Yield")
print("\nFirst few rows:")
print(result_df.head())

# Check for any missing values
print(f"\nMissing values in new columns:")
print(result_df[['Event_Date_AdjClose', 'Prior_Day_AdjClose', 'Dividend_Yield']].isnull().sum())

# Show some examples to verify the logic
print("\nSample verification (showing announcement date and most recent trading date):")
sample = result_df.head(3).copy()
for _, row in sample.iterrows():
    ticker = row['Ticker']
    ann_date = row['Announcement_Date']
    recent_dates = [d for d in available_dates[ticker] if d < ann_date]
    most_recent = max(recent_dates) if recent_dates else None
    print(f"Ticker: {ticker}, Announcement: {ann_date.date()}, Most Recent Trading: {most_recent.date() if most_recent else 'None'}")
merged_df.to_csv('merged_dividend_car_data.csv', index=False)

Final DataFrame Shape: (657, 8)

Columns in final dataset:
['Ticker', 'Announcement_Date', 'CAR', 'DPS', 'Dividend_Type', 'Event_Date_AdjClose', 'Prior_Day_AdjClose', 'Dividend_Yield']

New columns added: Event_Date_AdjClose, Prior_Day_AdjClose, Dividend_Yield

First few rows:
        Ticker Announcement_Date       CAR  DPS Dividend_Type  \
0  ADANIENT.NS        2016-03-09 -0.016032  0.4       Interim   
1  ADANIENT.NS        2017-05-24 -0.108284  0.4         Final   
2  ADANIENT.NS        2018-05-10 -0.039030  0.4         Final   
3  ADANIENT.NS        2019-05-29 -0.000906  0.4         Final   
4  ADANIENT.NS        2020-03-12 -0.135723  1.0       Interim   

   Event_Date_AdjClose  Prior_Day_AdjClose  Dividend_Yield  
0            35.040146           33.800018        0.011834  
1            60.596619           60.356586        0.006627  
2            79.972527           81.487999        0.004909  
3           153.172226          155.640350        0.002570  
4           160.570099    

In [9]:
result_df.to_csv('dividend_yield_analysis.csv', index=False)

### Dividend_Change_Pct

In [13]:
# Calculate dividend change for same dividend type
result_df['DPS_Change'] = result_df.groupby(['Ticker', 'Dividend_Type'])['DPS'].diff()

# Save to CSV
result_df.to_csv('dividend_yield_analysis_final.csv', index=False)

print("Final dataset created!")
print("Columns available:")
print(result_df.columns.tolist())
print(f"\nDPS_Change statistics:")
print(result_df['DPS_Change'].describe())

Final dataset created!
Columns available:
['Ticker', 'Announcement_Date', 'CAR', 'DPS', 'Dividend_Type', 'Event_Date_AdjClose', 'Prior_Day_AdjClose', 'Dividend_Yield', 'DPS_Change', 'DPS_Pct_Change', 'Dividend_Direction_Text', 'Dividend_Direction_Num']

DPS_Change statistics:
count    577.000000
mean       1.080364
std       18.244433
min     -133.000000
25%        0.000000
50%        0.250000
75%        2.400000
max      157.000000
Name: DPS_Change, dtype: float64


In [14]:
# Calculate dividend change for same dividend type
result_df['DPS_Change'] = result_df.groupby(['Ticker', 'Dividend_Type'])['DPS'].diff()

# Calculate market percentage change (assuming you have a 'Price' column)
result_df['DPS_Pct_Change'] = result_df.groupby('Ticker')['DPS'].pct_change()

# Save dataset without direction columns
result_df.to_csv('dividend_analysis_complete.csv', index=False)

print("Complete dataset created!")
print(f"Total rows: {len(result_df)}")
print("\nColumns available:")
print(result_df.columns.tolist())
print(f"\nDPS_Change statistics:")
print(result_df['DPS_Change'].describe())

Complete dataset created!
Total rows: 657

Columns available:
['Ticker', 'Announcement_Date', 'CAR', 'DPS', 'Dividend_Type', 'Event_Date_AdjClose', 'Prior_Day_AdjClose', 'Dividend_Yield', 'DPS_Change', 'DPS_Pct_Change', 'Dividend_Direction_Text', 'Dividend_Direction_Num']

DPS_Change statistics:
count    577.000000
mean       1.080364
std       18.244433
min     -133.000000
25%        0.000000
50%        0.250000
75%        2.400000
max      157.000000
Name: DPS_Change, dtype: float64


# Calculate dividend change for same dividend type
result_df['DPS_Change'] = result_df.groupby(['Ticker', 'Dividend_Type'])['DPS'].diff()

# Create both text and numeric direction columns
result_df['Dividend_Direction_Text'] = result_df['DPS_Change'].apply(
    lambda x: 'Increase' if x > 0 else ('Decrease' if x < 0 else 'No Change') if pd.notnull(x) else None
)

result_df['Dividend_Direction_Num'] = result_df['DPS_Change'].apply(
    lambda x: 1 if x > 0 else (-1 if x < 0 else 0) if pd.notnull(x) else None
)

# Drop rows where direction is None (first events in dataset)
div_change_df = result_df.dropna(subset=['Dividend_Direction_Num']).copy()

# Save dataset with both columns
div_change_df.to_csv('dividend_analysis_complete.csv', index=False)

print("Complete dataset created!")
print(f"Original rows: {len(result_df)}")
print(f"Rows after dropping first events: {len(regression_df)}")
print("\nDividend direction distribution:")
print(regression_df['Dividend_Direction_Text'].value_counts())
print(regression_df['Dividend_Direction_Num'].value_counts().sort_index())

In [49]:
div_change_df.to_csv('dividend_analysis_complete.csv', index=False)

In [17]:
# First, remove any existing ROA and Market Cap columns to avoid conflicts
columns_to_remove = ['ROA', 'ROA_x', 'ROA_y', 'Market_Cap_Cr', 'Market_Cap_Cr_x', 'Market_Cap_Cr_y']
for col in columns_to_remove:
    if col in regression_df.columns:
        regression_df = regression_df.drop(col, axis=1)

# Convert ROA and Market Cap data from wide to long format USING TICKER
roa_long = pd.melt(roa_df, id_vars=['Ticker'], value_vars=year_columns, var_name='FY_Year', value_name='ROA')
roa_long['FY_Year'] = roa_long['FY_Year'].astype(int)

mcap_long = pd.melt(mcap_df, id_vars=['Ticker'], value_vars=year_columns, var_name='FY_Year', value_name='Market_Cap_Cr')
mcap_long['FY_Year'] = mcap_long['FY_Year'].astype(int)

# Create relevant FY column in main dataset
def get_relevant_fy(announcement_date):
    year = announcement_date.year
    month = announcement_date.month
    return year if month >= 4 else year - 1

regression_df['Relevant_FY_Year'] = regression_df['Announcement_Date'].apply(get_relevant_fy)

# Merge ROA and Market Cap using TICKER (not Company_Name)
# First merge with ROA
regression_df = pd.merge(
    regression_df,
    roa_long,
    left_on=['Ticker', 'Relevant_FY_Year'],
    right_on=['Ticker', 'FY_Year'],
    how='left'
)

# Then merge with Market Cap
regression_df = pd.merge(
    regression_df,
    mcap_long,
    left_on=['Ticker', 'Relevant_FY_Year'],
    right_on=['Ticker', 'FY_Year'],
    how='left',
    suffixes=('', '_mcap')
)

# Clean up extra columns - check what columns actually exist before dropping
columns_to_drop = []
for col in ['FY_Year', 'FY_Year_roa', 'FY_Year_mcap']:
    if col in regression_df.columns:
        columns_to_drop.append(col)

if columns_to_drop:
    regression_df = regression_df.drop(columns_to_drop, axis=1)

# Alternative: drop columns that end with these patterns
# regression_df = regression_df.loc[:, ~regression_df.columns.str.endswith(('_x', '_y', '_roa', '_mcap'))]

# Save final dataset
regression_df.to_csv('complete_dividend_analysis_dataset.csv', index=False)

print("Merging completed successfully!")
print(f"Final dataset shape: {regression_df.shape}")
print(f"Columns: {list(regression_df.columns)}")

NameError: name 'regression_df' is not defined

In [114]:
# 1. Drop duplicate columns
if 'Dividend_Direction' in regression_df.columns:
    regression_df = regression_df.drop('Dividend_Direction', axis=1)

# 2. Drop any FY_Year_x, FY_Year_y columns
fy_columns_to_drop = ['FY_Year_x', 'FY_Year_y', 'FY_Year_mcap', 'ROA_FY_Year', 'MCap_FY_Year',  'Company_Name_x', 'Company_Name_y','Market_Cap_Cr_mcap', 'Log_Market_Cap_Cr','Market_Pct_Change']
for col in fy_columns_to_drop:
    if col in regression_df.columns:
        regression_df = regression_df.drop(col, axis=1)

# 3. Rename Relevant_FY_Year to FY_Year
if 'Relevant_FY_Year' in regression_df.columns:
    regression_df = regression_df.rename(columns={'Relevant_FY_Year': 'FY_Year'})

# 4. Rearrange columns (simplified version)
essential_columns = [
    'Announcement_Date', 'Company_Name', 'Ticker', 'FY_Year', 'Dividend_Type',
    'DPS', 'DPS_Change', 'DPS_Pct_Change', 'Dividend_Direction_Text', 'Dividend_Direction_Num',
    'Event_Date_AdjClose', 'Prior_Day_AdjClose', 'Dividend_Yield',
    'ROA', 'Market_Cap_Cr', 'CAR'
]

# Keep only columns that exist
existing_columns = [col for col in essential_columns if col in regression_df.columns]
other_columns = [col for col in regression_df.columns if col not in existing_columns]
regression_df = regression_df[existing_columns + other_columns]

# 5. Save clean dataset
regression_df.to_csv('REGRESSION_DATASET.csv', index=False)

print("Duplicate columns removed!")
print("Final columns:", regression_df.columns.tolist())

Duplicate columns removed!
Final columns: ['Announcement_Date', 'Ticker', 'Dividend_Type', 'DPS', 'DPS_Change', 'DPS_Pct_Change', 'Dividend_Direction_Text', 'Dividend_Direction_Num', 'Event_Date_AdjClose', 'Prior_Day_AdjClose', 'Dividend_Yield', 'CAR']


In [16]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# ===== 1) Load data from CSV and define columns =====
df = pd.read_csv('REGRESSION_DATASET_updated.csv')  # Change filename if different

needed_cols = ['CAR','Dividend_Yield','DPS_Pct_Change','Market_Cap_Cr','ROA','Dividend_Type','Ticker']
missing = [c for c in needed_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# ===== 2) Clean / coerce numerics (handles %, commas, and ( ) negatives) =====
def to_numeric_clean(s):
    # Works for both numeric and object series
    if pd.api.types.is_numeric_dtype(s):
        return pd.to_numeric(s, errors='coerce')
    s = s.astype(str).str.strip()
    # detect percentage and negative-in-parentheses
    has_pct = s.str.contains('%', na=False)
    neg_paren = s.str.match(r'^\(.*\)$', na=False)

    # remove commas, %, parentheses
    cleaned = s.str.replace(',', '', regex=False)\
               .str.replace('%', '', regex=False)\
               .str.replace('(', '', regex=False)\
               .str.replace(')', '', regex=False)

    vals = pd.to_numeric(cleaned, errors='coerce')
    # apply signs/scale
    vals[neg_paren] = -vals[neg_paren]
    vals[has_pct] = vals[has_pct] / 100.0
    return vals

num_cols = ['CAR','Dividend_Yield','DPS_Change','Market_Cap_Cr','ROA']
for c in num_cols:
    df[c] = to_numeric_clean(df[c])

# ===== 3) Make Dividend_Type numeric dummy (0/1) if not already numeric =====
# Assumes common labels like 'Final'/'Interim' or similar; adjust mapping if needed.
if not pd.api.types.is_numeric_dtype(df['Dividend_Type']):
    type_map = {
        'final': 1, 'f': 1,
        'interim': 0, 'i': 0
    }
    # safe lowercasing for mapping
    dt_lower = df['Dividend_Type'].astype(str).str.strip().str.lower()
    # start with NaN then fill from mapping
    dt_num = pd.Series(np.nan, index=df.index)
    dt_num = dt_lower.map(type_map)
    # if some are already numeric-like strings, coerce them
    fallback_numeric = pd.to_numeric(df['Dividend_Type'], errors='coerce')
    dt_num = dt_num.fillna(fallback_numeric)
    df['Dividend_Type_num'] = dt_num
else:
    df['Dividend_Type_num'] = pd.to_numeric(df['Dividend_Type'], errors='coerce')

# ===== 4) Drop rows with missing essentials or non-finite =====
keep_cols = ['CAR','Dividend_Yield','DPS_Pct_Change','Market_Cap_Cr','ROA','Dividend_Type_num','Ticker']
df = df[keep_cols].copy()

# Remove inf/-inf
for c in keep_cols[:-1]:  # exclude Ticker
    df = df[np.isfinite(df[c])]

# Drop NaNs
df = df.dropna(subset=keep_cols)

# Ensure Ticker is present and usable for clustering
df['Ticker'] = df['Ticker'].astype(str)

# Sanity check: types
print("Data types after cleaning:")
print(df.dtypes)
print(f"\nFinal dataset shape: {df.shape}")

# ===== 5) Build regression matrices =====
y = df['CAR']
X = df[['Dividend_Yield','DPS_Pct_Change','Market_Cap_Cr','ROA','Dividend_Type_num']]
X = sm.add_constant(X)

# ===== 6) Fit OLS with clustered SEs by Ticker (fallback to HC3 if only one firm) =====
groups = df['Ticker']
if groups.nunique() > 1:
    model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': groups})
else:
    model = sm.OLS(y, X).fit(cov_type='HC3')

print("\n" + "="*50)
print("REGRESSION RESULTS")
print("="*50)
print(model.summary())

Data types after cleaning:
CAR                  float64
Dividend_Yield       float64
DPS_Pct_Change       float64
Market_Cap_Cr        float64
ROA                  float64
Dividend_Type_num      int64
Ticker                object
dtype: object

Final dataset shape: (577, 7)

REGRESSION RESULTS
                            OLS Regression Results                            
Dep. Variable:                    CAR   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     4.376
Date:                Thu, 04 Sep 2025   Prob (F-statistic):            0.00433
Time:                        08:01:44   Log-Likelihood:                 783.18
No. Observations:                 577   AIC:                            -1554.
Df Residuals:                     571   BIC:                            -1528.
Df Model:                           5                                    

C:\Users\Adit Chaudhari\anaconda3\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5, but rank is 4
  warnings.warn('covariance of constraints does not have full '


### Only Features in this CSV

In [129]:
feat_clean

,Ticker,Company,Event_Date,Dividend_Yield,Dividend_Change_Pct,Momentum_20d,Volatility_30d,Beta,DivType_Interim
0,ADANIENT.NS,Adani Enterprises Limited,2021-05-05,0.000794,0.000000,0.147261,0.043471,1.326688,False
1,ADANIENT.NS,Adani Enterprises Limited,2022-05-04,0.000428,0.000000,0.154828,0.020791,1.501592,False
2,ADANIENT.NS,Adani Enterprises Limited,2023-05-04,0.000653,0.200000,0.058887,0.028315,3.458845,False
3,ADANIENT.NS,Adani Enterprises Limited,2024-05-02,0.000426,0.083333,-0.043985,0.016967,1.732479,False
4,ADANIENT.NS,Adani Enterprises Limited,2025-05-02,0.000565,0.000000,-0.020903,0.021618,1.588385,False
...,...,...,...,...,...,...,...,...,...
363,WIPRO.NS,Wipro Limited,2022-03-25,0.017254,4.000000,0.082515,0.019121,0.951547,True
364,WIPRO.NS,Wipro Limited,2023-01-13,0.005296,-0.800000,-0.000391,0.011284,1.042780,True
365,WIPRO.NS,Wipro Limited,2024-01-12,0.004650,0.000000,0.070166,0.019029,1.289916,True
366,WIPRO.NS,Wipro Limited,2025-01-17,0.021661,5.000000,-0.079417,0.014270,1.184603,True


In [125]:
# Count NaNs per feature
print(feat_df[features].isna().sum())

# Inspect rows with NaNs
nan_rows = feat_df[feat_df[features].isna().any(axis=1)]
print(nan_rows.head(10))   # show first few problematic rows

Dividend_Yield         0
Dividend_Change_Pct    0
Momentum_20d           0
Volatility_30d         0
Beta                   0
DivType_Interim        0
dtype: int64
Empty DataFrame
Columns: [Ticker, Company, Event_Date, Dividend_Yield, Dividend_Change_Pct, Momentum_20d, Volatility_30d, Beta, DivType_Interim]
Index: []


### Machine Learning

### Logistic Regression

In [133]:
# ==== 0) Imports ====
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# ==== 1) Load data ====
features_df = pd.read_csv("Features_Dataset.csv")
car_df      = pd.read_csv("CAR_by_event.csv")

# Ensure dates are datetime
features_df["Event_Date"] = pd.to_datetime(features_df["Event_Date"], errors="coerce")
car_df["event_date"]      = pd.to_datetime(car_df["event_date"], errors="coerce")

# >>> FIX: align column names so merge keys match <<<
car_df = car_df.rename(columns={"event_date": "Event_Date"})

# ==== 2) Merge features + CAR on Ticker & Event_Date ====
ml = features_df.merge(
    car_df[["Ticker", "Event_Date", "CAR"]],
    on=["Ticker", "Event_Date"],
    how="inner"
).copy()

# Drop missing/zero CAR (optional to drop zeros)
ml = ml.dropna(subset=["CAR"])
ml = ml[ml["CAR"] != 0]

# Label: +1 if CAR>0 else -1
ml["CAR_sign"] = np.where(ml["CAR"] > 0, 1, -1).astype(int)

# Ensure DivType_Interim is numeric 0/1
if ml["DivType_Interim"].dtype == bool:
    ml["DivType_Interim"] = ml["DivType_Interim"].astype(int)
elif ml["DivType_Interim"].dtype == object:
    ml["DivType_Interim"] = ml["DivType_Interim"].map({True:1, False:0, "True":1, "False":0}).fillna(ml["DivType_Interim"]).astype(float)

# ==== 3) Features (X) and Target (y) ====
features = ["Dividend_Yield","Dividend_Change_Pct","Momentum_20d","Volatility_30d","Beta","DivType_Interim"]
X = ml[features].copy()
y = ml["CAR_sign"]

# ==== 4) Train/Test split ====
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

# ==== 5) Pipeline: Impute → Scale → Logistic Regression ====
pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000, class_weight="balanced", solver="lbfgs"))
])
pipe.fit(X_train, y_train)

# ==== 6) Evaluation ====
y_pred = pipe.predict(X_test)
print(f"Rows merged (nonzero CAR): {len(ml)}")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print("\nConfusion Matrix (rows=true, cols=pred; order: [ +1 , -1 ])")
print(confusion_matrix(y_test, y_pred, labels=[1,-1]))
print("\nClassification Report")
print(classification_report(y_test, y_pred, digits=3))

# Coefficients (on standardized features)
coef = pd.Series(pipe.named_steps["clf"].coef_.ravel(), index=features)
print("\nLogistic coefficients (standardized):")
print(coef.sort_values(ascending=False))


Rows merged (nonzero CAR): 368
Test Accuracy: 0.613

Confusion Matrix (rows=true, cols=pred; order: [ +1 , -1 ])
[[31 21]
 [22 37]]

Classification Report
              precision    recall  f1-score   support

          -1      0.638     0.627     0.632        59
           1      0.585     0.596     0.590        52

    accuracy                          0.613       111
   macro avg      0.611     0.612     0.611       111
weighted avg      0.613     0.613     0.613       111


Logistic coefficients (standardized):
DivType_Interim        0.326043
Dividend_Change_Pct    0.164113
Momentum_20d           0.142726
Dividend_Yield         0.101547
Beta                   0.077771
Volatility_30d         0.056029
dtype: float64


### Random Forrest

In [135]:
# ==== 0) Imports ====
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# ==== 1) Load & merge (same keys as before) ====
features_df = pd.read_csv("Features_Dataset.csv")
car_df      = pd.read_csv("CAR_by_event.csv")

# Dates to datetime
features_df["Event_Date"] = pd.to_datetime(features_df["Event_Date"], errors="coerce")
car_df["event_date"]      = pd.to_datetime(car_df["event_date"], errors="coerce")

# Align column names
car_df = car_df.rename(columns={"event_date": "Event_Date"})

# Merge features + CAR
ml = features_df.merge(
    car_df[["Ticker", "Event_Date", "CAR"]],
    on=["Ticker", "Event_Date"],
    how="inner"
).copy()

# Keep valid, nonzero CAR and create label
ml = ml.dropna(subset=["CAR"])
ml = ml[ml["CAR"] != 0]
ml["CAR_sign"] = np.where(ml["CAR"] > 0, 1, -1).astype(int)

# Ensure dummy is numeric
if ml["DivType_Interim"].dtype == bool:
    ml["DivType_Interim"] = ml["DivType_Interim"].astype(int)

# ==== 2) Features & target ====
features = ["Dividend_Yield","Dividend_Change_Pct","Momentum_20d","Volatility_30d","Beta","DivType_Interim"]
X = ml[features].copy()
y = ml["CAR_sign"]

# ==== 3) Train/Test split ====
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

# ==== 4) Pipeline: Impute → RandomForest ====
rf_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("rf", RandomForestClassifier(
        n_estimators=500,
        max_depth=None,
        min_samples_leaf=2,
        class_weight="balanced",
        random_state=42,
        n_jobs=-1
    ))
])

# Fit
rf_pipe.fit(X_train, y_train)

# ==== 5) Test evaluation ====
y_pred = rf_pipe.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm  = confusion_matrix(y_test, y_pred, labels=[1,-1])

print(f"Rows merged (nonzero CAR): {len(ml)}")
print(f"Random Forest Test Accuracy: {acc:.3f}")
print("\nConfusion Matrix (rows=true, cols=pred; order: [ +1 , -1 ])")
print(cm)
print("\nClassification Report")
print(classification_report(y_test, y_pred, digits=3))

# ==== 6) 5-fold CV accuracy ====
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(rf_pipe, X, y, cv=cv, scoring="accuracy", n_jobs=-1)
print(f"\n5-fold CV Accuracy: mean={cv_scores.mean():.3f}, std={cv_scores.std():.3f}")
print("CV fold scores:", np.round(cv_scores, 3))

# ==== 7) Feature importances ====
# Pull trained RF out of the pipeline
rf = rf_pipe.named_steps["rf"]
importances = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=False)
print("\nRandom Forest feature importances:")
print(importances)

Rows merged (nonzero CAR): 368
Random Forest Test Accuracy: 0.514

Confusion Matrix (rows=true, cols=pred; order: [ +1 , -1 ])
[[25 27]
 [27 32]]

Classification Report
              precision    recall  f1-score   support

          -1      0.542     0.542     0.542        59
           1      0.481     0.481     0.481        52

    accuracy                          0.514       111
   macro avg      0.512     0.512     0.512       111
weighted avg      0.514     0.514     0.514       111


5-fold CV Accuracy: mean=0.533, std=0.046
CV fold scores: [0.608 0.473 0.5   0.548 0.534]

Random Forest feature importances:
Dividend_Yield         0.215651
Momentum_20d           0.205141
Volatility_30d         0.202286
Beta                   0.187830
Dividend_Change_Pct    0.144594
DivType_Interim        0.044497
dtype: float64
